# Semantičko jezgro s integracijom OpenBnB MCP poslužitelja

Ova bilježnica pokazuje kako koristiti Semantičko jezgro s pravim OpenBnB MCP poslužiteljem za pretraživanje stvarnih Airbnb smještaja koristeći MCPStdioPlugin. Za pristup LLM-u koristi Azure AI Foundry. Za postavljanje varijabli okruženja, možete slijediti [Lekciju za postavljanje](/00-course-setup/README.md)


## Uvezi potrebne pakete


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Kreiranje veze za MCP Plugin

Povezat ćemo se s [OpenBnB MCP serverom](https://github.com/openbnb-org/mcp-server-airbnb) koristeći MCPStdioPlugin. Ovaj server omogućuje funkcionalnost pretraživanja Airbnb-a putem paketa @openbnb/mcp-server-airbnb.


## Kreiranje klijenta

U ovom primjeru koristit ćemo Azure AI Foundry za pristup LLM-u. Provjerite jesu li vaše varijable okruženja ispravno postavljene.


## Konfiguracija okruženja

Postavite Azure OpenAI postavke. Provjerite imate li postavljene sljedeće varijable okruženja:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Razumijevanje integracije OpenBnB MCP-a

Ova bilježnica povezuje se s **pravim OpenBnB MCP serverom** koji pruža stvarnu funkcionalnost pretraživanja Airbnb-a.

### Kako funkcionira:

1. **MCPStdioPlugin**: Koristi komunikaciju putem standardnog ulaza/izlaza s MCP serverom
2. **Pravi NPM paket**: Preuzima i pokreće `@openbnb/mcp-server-airbnb` putem npx-a
3. **Podaci uživo**: Vraća stvarne podatke o Airbnb nekretninama iz njihovih API-ja
4. **Otkrivanje funkcija**: Agent automatski otkriva dostupne funkcije s MCP servera

### Dostupne funkcije:

OpenBnB MCP server obično pruža:
- **search_listings** - Pretraživanje Airbnb nekretnina prema lokaciji i kriterijima
- **get_listing_details** - Dobivanje detaljnih informacija o određenim nekretninama
- **check_availability** - Provjera dostupnosti za određene datume
- **get_reviews** - Dohvaćanje recenzija za nekretnine
- **get_host_info** - Dobivanje informacija o domaćinima nekretnina

### Preduvjeti:

- **Node.js** instaliran na vašem sustavu
- **Internetska veza** za preuzimanje MCP server paketa
- **NPX** dostupan (dolazi s Node.js-om)

### Testiranje veze:

Možete ručno testirati MCP server pokretanjem:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Ovo će preuzeti i pokrenuti OpenBnB MCP server, na koji se Semantic Kernel zatim povezuje za stvarne podatke o Airbnb-u.


## Pokretanje Agenta s OpenBnB MCP Serverom

Sada ćemo pokrenuti AI Agenta koji se povezuje s OpenBnB MCP serverom kako bi pretražio stvarne Airbnb smještaje u Stockholmu za 2 odrasle osobe i 1 dijete. Slobodno promijenite popis `user_inputs` kako biste prilagodili kriterije pretraživanja.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Sažetak  
Čestitamo! Uspješno ste izradili AI agenta koji se integrira s pretraživanjem smještaja u stvarnom svijetu koristeći Model Context Protocol (MCP):

Korištene tehnologije:  
- Semantic Kernel - Za izradu inteligentnih agenata s Azure OpenAI  
- Azure AI Foundry - Za mogućnosti LLM-a i dovršavanje razgovora  
- MCP (Model Context Protocol) - Za standardiziranu integraciju alata  
- OpenBnB MCP Server - Za stvarnu funkcionalnost pretraživanja Airbnb-a  
- Node.js/NPX - Za pokretanje vanjskog MCP servera  

Što ste naučili:  
- MCP integracija: Povezivanje Semantic Kernel agenata s vanjskim MCP serverima  
- Pristup podacima u stvarnom vremenu: Pretraživanje stvarnih Airbnb nekretnina putem live API-ja  
- Komunikacija protokola: Korištenje stdio komunikacije između agenta i MCP servera  
- Otkrivanje funkcija: Automatsko otkrivanje dostupnih funkcija s MCP servera  
- Streaming odgovora: Hvatanje i bilježenje poziva funkcija u stvarnom vremenu  
- HTML renderiranje: Formatiranje odgovora agenta sa stiliziranim tablicama i interaktivnim prikazima  

Sljedeći koraci:  
- Integrirajte dodatne MCP servere (vrijeme, letovi, restorani)  
- Izgradite sustav s više agenata koji kombinira MCP i A2A protokole  
- Kreirajte prilagođene MCP servere za vlastite izvore podataka  
- Implementirajte trajnu memoriju razgovora kroz sesije  
- Implementirajte agenta na Azure Functions s orkestracijom MCP servera  
- Dodajte autentifikaciju korisnika i mogućnosti rezervacije  

Ključne prednosti MCP arhitekture:  
- Standardizacija: Univerzalni protokol za povezivanje AI agenata s vanjskim alatima  
- Podaci u stvarnom vremenu: Pristup aktualnim, ažuriranim informacijama iz raznih usluga  
- Proširivost: Jednostavna integracija novih izvora podataka i alata  
- Interoperabilnost: Radi na različitim AI okvirima i platformama za agente  
- Odvajanje odgovornosti: Jasna razlika između AI logike i pristupa vanjskim podacima  



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
